In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [2]:
def extract_text_from_cf_problem_page(problem_url):
    problem_html = requests.get(problem_url)
    problem_parse = BeautifulSoup(problem_html.content,"html.parser",from_encoding="utf-8")
    problem_div = problem_parse.find_all("div")
    #問題文の抽出,コドフォのページのhtmlをよく見て該当部分の属性で抽出
    res={"problem_text":[],"input_text":[],"output_text":[],"tags":[]}
    for elem in problem_div:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "input-specification"):
                res["input_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
            if(elem_cls == "output-specification"):
                res["output_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])
                break
        except:
            if(not res["problem_text"]):
                res["problem_text"].extend([paragraph.string for paragraph in elem.find_all("p") if paragraph.string is not None])

    res["problem_text"]="".join(res["problem_text"])
    res["input_text"]="".join(res["input_text"])
    res["output_text"]="".join(res["output_text"])
    #タグの抽出,コドフォのhtmlをよく見て該当部分の属性で抽出
    problem_span = problem_parse.find_all("span")
    for elem in problem_span:
        try:
            elem_cls = elem.get("class")[0]
            if(elem_cls == "tag-box"):
                res["tags"].append(elem.string)
        except:
            pass
    res["tags"]="".join(res["tags"])
    assert type(res["problem_text"])==type(res["input_text"])==type(res["output_text"])==type(res["tags"])==str, "output is not strings"
    return res

In [3]:
base_url = "https://codeforces.com/problemset/problem/"
prob_nums = [str(i) for i in range(200,1500)] #200回から最新まで抽出
capital = [chr(i) for i in range(65, 65+9)] #ABCDEFGHI
prob_names = capital + [e+"1" for e in capital] #easyとhardがある場合があるがhardは割愛(問題文がほぼ同じため)

In [4]:
df = {"prob_id":[],"problem_texts":[],"input_texts":[],"output_texts":[],"tag_set":[]}

In [ ]:
save_point=0
for num in tqdm(prob_nums):
    for name in prob_names:
        prob_id = num+"/"+name
        try:
            datum = extract_text_from_cf_problem_page(base_url+prob_id)
            df["prob_id"].append(prob_id)
            df["problem_texts"].append(datum["problem_text"])
            df["input_texts"].append(datum["input_text"])
            df["output_texts"].append(datum["output_text"])
            df["tag_set"].append(datum["tags"])
        except:
            pass
        assert len(df["prob_id"])==len(df["problem_texts"])==len(df["input_texts"])==len(df["output_texts"])==len(df["tag_set"]), "does not the same length"
    if(save_point%10==0):
        pd.DataFrame(df).to_csv("../data/cf_problem_tag_dataset.csv",index=False)
    save_point+=1

 68%|██████▊   | 889/1300 [9:36:22<3:59:15, 34.93s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 68%|██████▊   | 890/1300 [9:37:05<4:15:53, 37.45s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were repla

In [ ]:
pd.DataFrame(df).to_csv("../data/cf_problem_tag_dataset.csv",index=False)